In [1]:
! pip install streamlit -q

     |████████████████████████████████| 9.2 MB 7.8 MB/s 
     |████████████████████████████████| 4.7 MB 46.5 MB/s 
     |████████████████████████████████| 182 kB 50.0 MB/s 
     |████████████████████████████████| 237 kB 47.0 MB/s 
     |████████████████████████████████| 78 kB 7.6 MB/s 
     |████████████████████████████████| 164 kB 59.9 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 51 kB 7.5 MB/s 


Write the cell python code into an app.py file

In [2]:
! pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
 ! pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 8.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=7e9427c579f3b25af91eb13327598a867fc55ea35eca26a359028db6c58e2de0
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [4]:
from pyngrok import ngrok

In [5]:
ngrok.set_auth_token("2GvyHNIBPlWzl6MBWa9MTkrc5Zi_5bzwCPJSJoW6dmycUnPTB") #ngrok.com

In [6]:
!pip install yfinance prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.4 MB/s 
     |████████████████████████████████| 4.0 MB 15.9 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 128 kB 66.4 MB/s 
     |████████████████████████████████| 110 kB 65.6 MB/s 
     |████████████████████████████████| 112 kB 62.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [7]:
%%writefile app.py
# Data processing
import streamlit as st
import cufflinks as cf
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Get time series data
import yfinance as yf

# Prophet model for time series forecast
from prophet import Prophet
from prophet.plot import plot_plotly, add_changepoints_to_plot, plot_cross_validation_metric
from prophet.diagnostics import cross_validation, performance_metrics
from datetime import date

# Visualization
import plotly.graph_objs as go


#Stockmarket Price
# App title
st.markdown('''
# Stock Market Forecasting App
- Maulana Firman Nurdiansyah
- Rizky Firman Ardiansyah
''')
st.write('---')

# Sidebar
st.sidebar.subheader('Query parameters')
start_date = st.sidebar.date_input("Start date", datetime.date(2020, 11, 29))
end_date = st.sidebar.date_input("End date", datetime.date(2022, 11, 29))

# Retrieving tickers data
ticker_list = pd.read_csv('/content/Ticker Stockmarket forecasting 1325 company.txt')
tickerSymbol = st.sidebar.selectbox('Stock ticker', ticker_list) # Select ticker symbol
tickerData = yf.Ticker(tickerSymbol) # Get ticker data
tickerDf = tickerData.history(period='1d', start=start_date, end=end_date) #get the historical prices for this ticker

# Ticker information
string_logo = '<img src=%s>' % tickerData.info['logo_url']
st.markdown(string_logo, unsafe_allow_html=True)

string_name = tickerData.info['longName']
st.header('**%s**' % string_name)

string_summary = tickerData.info['longBusinessSummary']
st.info(string_summary)

# Ticker data
st.header('**Ticker data**')
st.write(tickerDf)

# Bollinger bands
st.header('**Bollinger Bands**')
qf=cf.QuantFig(tickerDf,title='First Quant Figure',legend='top',name='GS')
qf.add_bollinger_bands()
fig = qf.iplot(asFigure=True)
st.plotly_chart(fig)

####
#st.write('---')
#st.write(tickerData.info)

#Stockmarket Forecasting
n_day = st.slider('Days of prediction:', 1, 60)
period = n_day

def load_data(ticker):
    data = yf.download(ticker, start_date, end_date)
    data.reset_index(inplace=True)
    return data

data_load_state = st.text('Loading data...')
data = load_data(tickerSymbol)
data_load_state.text('Loading data... done!')

st.subheader('Raw data')
st.write(data.tail())

# Plot raw data
def plot_raw_data():
	fig = go.Figure()
	fig.add_trace(go.Scatter(x=data['Date'], y=data['Open'], name="stock_open"))
	fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name="stock_close"))
	fig.layout.update(title_text='Time Series data with Rangeslider', xaxis_rangeslider_visible=True)
	st.plotly_chart(fig)

plot_raw_data()

# Predict forecast with Prophet.
df_train = data[['Date','Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)

st.write(f'Forecast plot for {n_day} days')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

FinalForecast = forecast[["ds","trend"]]
FinalForecast = FinalForecast.rename(columns={"ds": "Tanggal", "trend": "harga"})
st.write(FinalForecast.tail(n_day))

st.write("Forecast components")
fig2 = m.plot_components(forecast)
st.write(fig2)

# Cross validation
df_cv = cross_validation(m, initial='500 days', period='60 days', horizon = '60 days', parallel="processes")
st.write(df_cv)

# Model performance metrics
df_p = performance_metrics(df_cv)

# Visualize the performance metrics
st.write("Visualize the performance metrics")
fig3 = plot_cross_validation_metric(df_cv, metric='mape')
st.write(fig3)

fig4 = plot_cross_validation_metric(df_cv, metric='rmse')
st.write(fig4)

fig5 = plot_cross_validation_metric(df_cv, metric='mse')
st.write(fig5)

fig6 = plot_cross_validation_metric(df_cv, metric='mae')
st.write(fig6)

fig7 = plot_cross_validation_metric(df_cv, metric='mdape')
st.write(fig7)

fig8 = plot_cross_validation_metric(df_cv, metric='smape')
st.write(fig8)


Writing app.py


In [9]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://3ad9-34-125-92-113.ngrok.io" -> "http://localhost:80"
